Copyright **`(c)`** 2023 Thomas Baracco `<s308722@studenti.polito.it>`  
[`https://github.com/baraccothomas/computational-intelligence`](https://github.com/baraccothomas/computational-intelligence)  
Free for personal or classroom use.

In [123]:
import numpy as np

from random import random
from collections import namedtuple
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue


In [124]:
PROBLEM_SIZE = 40
NUM_SETS = 50
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

State = namedtuple('State', ['taken', 'not_taken'])

In [125]:
class Node:
    def __init__(self, state, g, h):
        self.state = state
        self.g = g
        self.h = h
    
    def f(self) -> np.int64:
        return self.g + self.h

    def __str__(self):
        return f"State: {self.state} | g = {self.g} | h = {self.h} | f = {self.f()}"
    
    def __eq__(self, other):
        return (self.state == other.state) and (self.g == other.g)

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return (self.state < other.state) and (self.g < other.g)

    def __gt__(self, other):
        return (self.state > other.state) and (self.g > other.g)

    def __le__(self, other):
        return (self < other) or (self == other)

    def __ge__(self, other):
        return (self > other) or (self == other)

In [126]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for i in range(PROBLEM_SIZE)]),
        )
    )


def h(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [127]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [128]:
frontier = PriorityQueue()
initialState = State(set(), set(range(NUM_SETS)))
initialNode = Node(initialState, 0, h(initialState))
frontier.put((initialNode.f(), initialNode))

counter = 0
_, current_node = frontier.get()

while not goal_check(current_node.state):
    counter += 1
    for action in current_node.state[1]:
        new_state = State(
            current_node.state.taken ^ {action},
            current_node.state.not_taken ^ {action},
        )
        new_node = Node(new_state, current_node.g + 1, current_node.h + h(new_state))
        frontier.put((new_node.f(), new_node))

    
    _, current_node = frontier.get()

print(current_node.state)

print(
    f"Solved in {counter:,} steps ({len(current_node.state.taken)} tiles)"
)

State(taken={0, 33, 38, 19, 21}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49})
Solved in 175 steps (5 tiles)
